In this notebook I am going to calculate the average price of the sale of the houses based on the following conditions:
    - If the house has more than 2 rooms, I segment by:
        - more than 11,500 square feet.
        - equal or less than 11,500 square feet.
        
    - If the house has 2 rooms or less, I segment by :
        - more than 8,500 square feet.
        - equal or less than 8,500 square feet

In [1]:
import pandas as pd
import numpy as np

#### First i'm going to load the 'train.csv' and ckeck the dataframe

In [2]:
#load training set
training_set = pd.read_csv('train.csv')
pd.options.display.max_columns = None
training_set.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
1143,1144,20,RL,NaN,9000,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Norm,Norm,1Fam,1Story,5,3,1959,1959,Gable,CompShg,Wd Sdng,Plywood,None,0.0,TA,TA,CBlock,TA,TA,No,GLQ,288,Unf,0,718,1006,GasA,TA,Y,SBrkr,1006,0,0,1006,0,0,1,0,3,1,TA,5,Typ,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,Y,0,24,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal,80000
372,373,120,RL,50.0,7175,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,SawyerW,Norm,Norm,TwnhsE,1Story,6,5,1984,1984,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,No,ALQ,623,LwQ,121,0,744,GasA,TA,Y,SBrkr,752,0,0,752,1,0,1,0,2,1,TA,4,Typ,0,NaN,Attchd,1984.0,Unf,1,264,TA,TA,Y,353,0,0,0,90,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,125000
334,335,60,RL,59.0,9042,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1998,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,Gd,GLQ,828,Unf,0,115,943,GasA,Gd,Y,SBrkr,943,695,0,1638,1,0,2,1,3,1,TA,7,Typ,2,TA,Attchd,1998.0,Fin,2,472,TA,TA,Y,100,38,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal,192000
68,69,30,RM,47.0,4608,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Artery,Norm,1Fam,1Story,4,6,1945,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,Gd,CBlock,TA,TA,No,Unf,0,Unf,0,747,747,GasA,TA,Y,SBrkr,747,0,0,747,0,0,1,0,2,1,TA,4,Typ,0,NaN,Attchd,1945.0,Unf,1,220,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2010,WD,Normal,80000
494,495,30,RM,50.0,5784,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,OldTown,Artery,Norm,1Fam,1Story,5,8,1938,1996,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,Fa,TA,No,Unf,0,Unf,0,190,190,GasA,Gd,Y,FuseA,886,0,0,886,0,0,1,0,2,1,TA,4,Typ,0,NaN,Attchd,1938.0,Unf,1,273,TA,TA,Y,144,20,80,0,0,0,NaN,NaN,NaN,0,12,2009,WD,Normal,91300


#### I create field to report if there are equal or less than 2 rooms or if there are more than 2 rooms

In [4]:
training_set['bedrooms_group'] = training_set['BedroomAbvGr'].apply(lambda x: '+2' if x > 2 else '2 or less')
training_set.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,bedrooms_group
1003,1004,90,RL,NaN,11500,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NWAmes,Feedr,RRAn,Duplex,1Story,5,6,1976,1976,Gable,CompShg,VinylSd,VinylSd,BrkFace,164.0,TA,TA,CBlock,TA,TA,No,Unf,0,Unf,0,1680,1680,GasA,Fa,Y,SBrkr,1680,0,0,1680,0,0,2,0,4,2,TA,8,Typ,0,NaN,Detchd,1976.0,Unf,2,528,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2007,WD,Normal,136905,+2
81,82,120,RM,32.0,4500,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Mitchel,Norm,Norm,TwnhsE,1Story,6,5,1998,1998,Hip,CompShg,VinylSd,VinylSd,BrkFace,443.0,TA,Gd,PConc,Ex,Gd,No,GLQ,1201,Unf,0,36,1237,GasA,Ex,Y,SBrkr,1337,0,0,1337,1,0,2,0,2,1,TA,5,Typ,0,NaN,Attchd,1998.0,Fin,2,405,TA,TA,Y,0,199,0,0,0,0,NaN,NaN,NaN,0,3,2006,WD,Normal,153500,2 or less
1044,1045,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,PosN,Norm,1Fam,1Story,8,5,1981,1981,Hip,WdShngl,BrkFace,BrkFace,None,0.0,Gd,TA,PConc,Gd,TA,No,ALQ,1104,Unf,0,1420,2524,GasA,TA,Y,SBrkr,2524,0,0,2524,1,0,2,1,4,1,Gd,9,Typ,1,Gd,Attchd,1981.0,Fin,2,542,TA,TA,Y,474,120,0,0,0,0,NaN,MnPrv,NaN,0,7,2009,WD,Normal,278000,+2
918,919,60,RL,103.0,13125,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,SawyerW,Norm,Norm,1Fam,2Story,7,5,1991,1991,Gable,CompShg,HdBoard,HdBoard,None,0.0,Gd,TA,PConc,Ex,TA,Mn,BLQ,48,GLQ,634,422,1104,GasA,Ex,Y,SBrkr,912,1215,0,2127,1,0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1991.0,RFn,3,833,TA,TA,Y,72,192,224,0,0,0,NaN,GdPrv,NaN,0,11,2007,WD,Normal,238000,+2
1441,1442,120,RM,NaN,4426,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,TwnhsE,1Story,6,5,2004,2004,Gable,CompShg,VinylSd,VinylSd,BrkFace,147.0,Gd,TA,PConc,Gd,TA,Av,GLQ,697,Unf,0,151,848,GasA,Ex,Y,SBrkr,848,0,0,848,1,0,1,0,1,1,Gd,3,Typ,1,TA,Attchd,2004.0,RFn,2,420,TA,TA,Y,149,0,0,0,0,0,NaN,NaN,NaN,0,5,2008,WD,Normal,149300,2 or less


#### I divide the df into two dataframes: one with more than 2 rooms and another with 2 rooms or less.

In [5]:
more2bedrooms = training_set[training_set['bedrooms_group'] == '+2']
more2bedrooms.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,bedrooms_group
590,591,60,RL,64.0,8320,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,7,5,2004,2004,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,490,Unf,0,280,770,GasA,Ex,Y,SBrkr,770,812,0,1582,0,0,2,1,3,1,Gd,6,Typ,0,NaN,Attchd,2004.0,RFn,2,520,TA,TA,Y,0,45,0,0,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal,185900,+2
668,669,20,RL,NaN,14175,Pave,NaN,Reg,Bnk,AllPub,Corner,Mod,Sawyer,Norm,Norm,1Fam,1Story,5,6,1956,1987,Gable,CompShg,CemntBd,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,988,Unf,0,200,1188,GasA,Gd,Y,SBrkr,1437,0,0,1437,1,0,1,1,3,1,TA,6,Min2,1,TA,Detchd,1999.0,Unf,2,576,TA,TA,Y,304,0,0,0,0,0,NaN,NaN,NaN,0,11,2006,WD,Normal,168000,+2
888,889,20,RL,95.0,15865,Pave,NaN,IR1,Lvl,AllPub,Inside,Mod,NAmes,Norm,Norm,1Fam,1Story,8,6,1970,1970,Flat,Tar&Grv,Wd Sdng,Wd Sdng,None,0.0,Gd,Gd,PConc,TA,Gd,Gd,ALQ,351,Rec,823,1043,2217,GasA,Ex,Y,SBrkr,2217,0,0,2217,1,0,2,0,4,1,Gd,8,Typ,1,TA,Attchd,1970.0,Unf,2,621,TA,TA,Y,81,207,0,0,224,0,NaN,NaN,NaN,0,10,2007,WD,Normal,268000,+2
624,625,60,RL,80.0,10400,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,2Story,6,5,1972,1972,Gable,CompShg,VinylSd,VinylSd,None,288.0,TA,TA,CBlock,TA,TA,No,Rec,247,Unf,0,485,732,GasA,Gd,Y,SBrkr,1012,778,0,1790,1,0,1,2,4,1,TA,8,Min2,1,TA,Attchd,1972.0,RFn,2,484,TA,TA,Y,148,0,0,0,147,0,NaN,NaN,NaN,0,11,2006,WD,Normal,165150,+2
705,706,190,RM,70.0,5600,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,IDOTRR,Norm,Norm,2fmCon,2Story,4,5,1930,1950,Hip,CompShg,VinylSd,Wd Shng,None,0.0,Fa,Fa,Slab,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,Fa,N,SBrkr,372,720,0,1092,0,0,2,0,3,2,Fa,7,Mod,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,N,0,0,0,0,0,0,NaN,NaN,Othr,3500,7,2010,WD,Normal,55000,+2


In [6]:
lessEqual2bedrooms = training_set[training_set['bedrooms_group'] == '2 or less'] 
lessEqual2bedrooms.sample(5)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,bedrooms_group
1163,1164,90,RL,60.0,12900,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Sawyer,Feedr,Norm,Duplex,SFoyer,4,4,1969,1969,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,Av,GLQ,1198,Unf,0,0,1198,GasA,TA,Y,SBrkr,1258,0,0,1258,2,0,0,2,0,2,TA,6,Typ,0,NaN,CarPort,1969.0,Unf,2,400,Fa,TA,Y,120,0,0,0,0,0,NaN,NaN,NaN,0,1,2008,WD,Alloca,108959,2 or less
1089,1090,120,FV,37.0,3316,Pave,Pave,IR1,Lvl,AllPub,Inside,Gtl,Somerst,Norm,Norm,TwnhsE,1Story,8,5,2005,2005,Gable,CompShg,MetalSd,MetalSd,None,0.0,Gd,TA,PConc,Gd,TA,No,GLQ,1039,Unf,0,208,1247,GasA,Ex,Y,SBrkr,1247,0,0,1247,1,0,1,1,1,1,Gd,4,Typ,1,Gd,Attchd,2005.0,Fin,2,550,TA,TA,Y,0,84,0,0,0,0,NaN,NaN,NaN,0,4,2006,WD,Normal,197000,2 or less
426,427,80,RL,NaN,12800,Pave,NaN,Reg,Low,AllPub,Inside,Mod,SawyerW,Norm,Norm,1Fam,SLvl,7,5,1989,1989,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,145.0,Gd,TA,PConc,Gd,TA,Gd,GLQ,1518,Unf,0,0,1518,GasA,Gd,Y,SBrkr,1644,0,0,1644,1,1,2,0,2,1,Gd,5,Typ,1,TA,Attchd,1989.0,Fin,2,569,TA,TA,Y,80,0,0,0,396,0,NaN,NaN,NaN,0,8,2009,WD,Normal,275000,2 or less
901,902,20,RL,64.0,8712,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,5,7,1957,2000,Hip,CompShg,MetalSd,MetalSd,None,0.0,TA,Gd,CBlock,TA,TA,Mn,BLQ,860,Unf,0,132,992,GasA,TA,Y,SBrkr,1306,0,0,1306,1,0,1,0,2,1,TA,5,Typ,0,NaN,Detchd,1968.0,Unf,1,756,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,5,2009,WD,Normal,153000,2 or less
669,670,30,RL,80.0,11600,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1Story,4,5,1922,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,Fa,TA,No,Unf,0,Unf,0,700,700,GasA,Ex,Y,SBrkr,1180,0,0,1180,0,0,1,0,2,1,Fa,5,Typ,1,Gd,Detchd,1922.0,Unf,1,252,TA,Fa,Y,0,0,67,0,0,0,NaN,NaN,NaN,0,7,2006,WD,Normal,137500,2 or less


#### I create another field to report if there are equal or less than 11.500 square feet in the biggest houses or if there are more than 11.500 square feet

In [7]:
more2bedrooms['square_feet_group'] = more2bedrooms['LotArea'].apply(lambda x: '+11500' if x > 11500 else '11500 or less')
more2bedrooms.sample(5)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,bedrooms_group,square_feet_group
833,834,20,RL,100.0,10004,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1964,1964,Gable,CompShg,HdBoard,Plywood,BrkFace,180.0,TA,TA,CBlock,TA,TA,No,Rec,196,BLQ,345,975,1516,GasA,TA,Y,SBrkr,1516,0,0,1516,0,0,1,1,3,1,TA,6,Typ,0,NaN,Attchd,1964.0,RFn,2,472,TA,TA,Y,0,0,0,0,152,0,NaN,NaN,NaN,0,2,2009,WD,Normal,167000,+2,11500 or less
337,338,20,RL,70.0,9135,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,2002,2003,Gable,CompShg,VinylSd,VinylSd,BrkFace,113.0,Gd,TA,PConc,Gd,TA,Av,GLQ,810,Unf,0,726,1536,GasA,Ex,Y,SBrkr,1536,0,0,1536,1,0,2,0,3,1,Gd,7,Typ,0,NaN,Attchd,2002.0,RFn,2,532,TA,TA,Y,192,74,0,0,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal,214000,+2,11500 or less
1314,1315,20,RL,60.0,8190,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,4,6,1954,1954,Hip,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,TA,TA,No,Rec,732,Unf,0,216,948,GasA,Ex,Y,SBrkr,948,0,0,948,1,0,1,0,3,1,TA,5,Typ,1,TA,Detchd,1956.0,Unf,1,280,TA,TA,Y,0,36,0,0,0,0,NaN,NaN,NaN,0,10,2007,WD,Normal,119000,+2,11500 or less
1277,1278,80,RL,NaN,17871,Pave,NaN,IR1,Lvl,AllPub,CulDSac,Gtl,NWAmes,Norm,Norm,1Fam,SLvl,6,5,1967,1976,Gable,CompShg,HdBoard,HdBoard,BrkFace,359.0,TA,TA,CBlock,Gd,TA,Av,ALQ,528,Unf,0,1152,1680,GasA,Fa,Y,SBrkr,1724,0,0,1724,1,0,1,1,3,1,TA,7,Typ,1,Gd,Attchd,1967.0,RFn,2,480,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,6,2009,WD,Normal,197900,+2,+11500
487,488,20,RL,70.0,12243,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,NWAmes,Norm,Norm,1Fam,1Story,5,6,1971,1971,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,Gd,TA,Av,ALQ,998,Unf,0,486,1484,GasA,Gd,Y,SBrkr,1484,0,0,1484,0,0,2,0,3,1,TA,7,Typ,1,TA,Attchd,1971.0,Unf,2,487,TA,TA,Y,224,0,0,0,180,0,NaN,NaN,NaN,0,2,2007,WD,Normal,175000,+2,+11500


In [8]:
lessEqual2bedrooms['square_feet_group'] = lessEqual2bedrooms['LotArea'].apply(lambda x: '+8500' if x > 8500 else '8500 or less')
lessEqual2bedrooms.sample(5)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice,bedrooms_group,square_feet_group
821,822,20,RM,60.0,6000,Pave,Pave,Reg,Bnk,AllPub,Inside,Mod,OldTown,Norm,Norm,2fmCon,1Story,4,4,1953,1953,Gable,CompShg,MetalSd,MetalSd,None,0.0,Fa,TA,CBlock,Fa,TA,No,Unf,0,Unf,0,936,936,GasA,TA,N,SBrkr,936,0,0,936,0,0,1,0,2,1,TA,4,Min2,0,NaN,Detchd,1974.0,Unf,2,576,TA,TA,Y,0,32,112,0,0,0,NaN,NaN,NaN,0,2,2009,WD,Normal,93000,2 or less,8500 or less
710,711,30,RL,56.0,4130,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,BrkSide,Norm,Norm,1Fam,1Story,3,6,1935,2003,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,CBlock,TA,TA,No,Unf,0,Unf,0,270,270,GasA,Gd,Y,SBrkr,729,0,0,729,0,0,1,0,2,1,TA,5,Maj2,0,NaN,NaN,NaN,NaN,0,0,NaN,NaN,N,0,0,0,0,0,0,NaN,NaN,NaN,0,7,2008,WD,Normal,52000,2 or less,8500 or less
428,429,20,RL,64.0,6762,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,1Story,7,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,BrkFace,108.0,Gd,TA,PConc,Gd,TA,No,GLQ,664,Unf,0,544,1208,GasA,Ex,Y,SBrkr,1208,0,0,1208,1,0,2,0,2,1,Gd,6,Typ,0,NaN,Attchd,2007.0,RFn,2,628,TA,TA,Y,105,54,0,0,0,0,NaN,NaN,NaN,0,9,2007,New,Partial,195400,2 or less,8500 or less
736,737,90,RL,60.0,8544,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,Duplex,1Story,3,4,1950,1950,Gable,CompShg,Stucco,Stone,None,0.0,TA,TA,CBlock,NaN,NaN,NaN,NaN,0,NaN,0,0,0,GasA,Gd,N,FuseF,1040,0,0,1040,0,0,2,0,2,2,TA,6,Typ,0,NaN,Detchd,1949.0,Unf,2,400,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,7,2006,WD,Normal,93500,2 or less,+8500
771,772,20,RL,67.0,8877,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1Story,4,5,1951,1951,Gable,CompShg,Wd Sdng,Wd Sdng,None,0.0,TA,TA,CBlock,Fa,Fa,No,LwQ,836,Unf,0,0,836,GasA,TA,Y,FuseF,1220,0,0,1220,0,0,1,0,2,1,TA,6,Typ,0,NaN,Detchd,1951.0,Unf,2,396,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2006,COD,Normal,102000,2 or less,+8500


#### I'm going to get the average housing price for each of the 4 segmentations made

In [9]:
prices_more_2_rooms = more2bedrooms.groupby('square_feet_group')[['SalePrice']].mean()
prices_more_2_rooms

,SalePrice
square_feet_group,
+11500,233631.638436
11500 or less,170227.592693


In [10]:
prices_equal_less_2_rooms = lessEqual2bedrooms.groupby('square_feet_group')[['SalePrice']].mean()
prices_equal_less_2_rooms

,SalePrice
square_feet_group,
+8500,183796.030488
8500 or less,145917.172000
